<a href="https://colab.research.google.com/github/victorsa2/sales_meli/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparando o Ambiente no Github

In [ ]:
!git clone https://github.com/victorsa2/sales_meli.git

Cloning into 'sales_meli'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


## Subindo o arquivo .env com as variáveis no drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from google.colab import files
uploaded = files.upload()

Saving .env to .env (1)


##Instalando Dependencias não disponíveis no colab

In [7]:
pip install python-dotenv

In [ ]:
import requests
import webbrowser
from dotenv import load_dotenv
import os

##Consultando API do Mercado Livre

#Obter Access Token e User ID

In [7]:
def get_access_token_and_user_id():
    load_dotenv('/content/.env')

    APP_ID = os.getenv("APP_ID")
    APP_SECRET = os.getenv("APP_SECRET")
    REDIRECT_URI = os.getenv("REDIRECT_URI")

    # Cria a URL de autorização
    auth_url = f'https://auth.mercadolivre.com.br/authorization?response_type=code&client_id={APP_ID}&redirect_uri={REDIRECT_URI}'

    # Retorna a URL para que o usuário a abra no navegador
    return auth_url

def request_token(authorization_code):
    token_url = 'https://api.mercadolibre.com/oauth/token'
    payload = {
        'grant_type': 'authorization_code',
        'client_id': os.getenv("APP_ID"),
        'client_secret': os.getenv("APP_SECRET"),
        'code': authorization_code,
        'redirect_uri': os.getenv("REDIRECT_URI")
    }

    response = requests.post(token_url, data=payload)
    token_data = response.json()

    access_token = token_data['access_token']
    user_id = token_data['user_id']

    return access_token, user_id

if __name__ == "__main__":
    auth_url = get_access_token_and_user_id()
    print("Abra o seguinte URL no seu navegador para obter o código de autorização:")
    print(auth_url)

    authorization_code = input("Insira o código de autorização: ")
    access_token, user_id = request_token(authorization_code)


Abra o seguinte URL no seu navegador para obter o código de autorização:
https://auth.mercadolivre.com.br/authorization?response_type=code&client_id=5500572046072825&redirect_uri=https://victorsa.netlify.app
Insira o código de autorização: TG-66f985ed4aea6100011658cf-478295390


# Consumindo API por produtos para conseguir alguns vendedores de exemplo

In [ ]:
ACCESS_TOKEN = 'APP_USR-5500572046072825-092911-448787bf26cd8a408fe0949aa786dc7d-478295390'
SITE_ID = 'MLA'  # ou o site correspondente que você deseja usar
CATEGORY_ID = 'MLA1055'  # ID da categoria que você deseja buscar

# URL da API
url = f'https://api.mercadolibre.com/sites/{SITE_ID}/search?category={CATEGORY_ID}'

# Cabeçalhos da requisição
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

# Fazer a requisição GET
response = requests.get(url, headers=headers)

# Verificar o status da resposta
if response.status_code == 200:
    data = response.json()

    # Filtrar resultados e imprimir nickname do vendedor
    for item in data['results']:
        # Verifica se o item possui seller e nickname
        if 'seller' in item and 'nickname' in item['seller']:
            nickname = item['seller']['nickname']
            title = item['title']
            price = item['price']
            permalink = item['permalink']

            print(f'Título: {title}')
            print(f'Preço: {price}')
            print(f'Vendedor (Nickname): {nickname}')
            print(f'Link: {permalink}')
            print('-' * 40)
else:
    print(f'Erro na requisição: {response.status_code}')
